In [2]:
import torch
# model_silero_te, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
#                                                                   model='silero_te')
import re
import string
from nltk.tokenize import word_tokenize,TreebankWordTokenizer


In [3]:
import soundfile

# load model and processor
import os

# import librosa
import jiwer
import string
import csv
from autocorrect import Speller

spell = Speller()
from transformers import pipeline

# fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")
model_silero_te, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                  model='silero_te') # Light

def get_gold_hyp_lists(gold_path, test_path, septype):
    def create_hyp_sentence_dictionary(filename):
        sentence_dict = {}
        with open(filename, "r") as file:
            lines = file.readlines()
            for idx, line in enumerate(lines):
                line = line.strip()
                if line and len(line.split(septype, 1)) == 2:
                    # print(line.split(septype, 1))
                    sentence_id, sentence = line.split(septype, 1)
                    # sentence_id=sentence_id+".wav" # for AMI
                    sentence_id = sentence_id.split(".", 1)[0]  # cv
                    lm_out = apply_te(sentence.lower(), lan="en")
                    sentence_dict[sentence_id] = lm_out
        return sentence_dict

    def create_gold_sentence_dictionary(filename):
        sentence_dict = {}
        with open(filename, "r") as file:
            for line in file:
                # parts = line.split(' ', 1) # Libri
                parts = line.split(",", 1)  # AMI
                sentence_id, sentence = parts
                # print(sentence_id)
                # sentence_id = sentence_id.split("/", 1)[1]  # AMI
                sentence_id = sentence_id.split(".", 1)[0]  # cv
                # print(sentence_id)
                # print(sentence.strip())
                if sentence.strip() != "":
                    sentence_dict[sentence_id] = sentence.strip()
        return sentence_dict

    def read_csv_to_dict(file_path):
        result_dict = {}
        with open(file_path, "r") as csvfile:
            csv_reader = csv.reader(csvfile)
            for row in csv_reader:
                key, value = row[0], row[1]
                key = key.split("/")[1]
                # print(key,value)
                # break
                result_dict[key] = value
        return result_dict

    gold_dict = create_gold_sentence_dictionary(gold_path)
    # gold_dict = read_csv_to_dict(gold_path)
    test_dict = create_hyp_sentence_dictionary(test_path)

    gold_list = []
    test_list = []
    for test_key, test_value in test_dict.items():
        # print(test_key)
        if test_key in gold_dict:
            gold = gold_dict[test_key]
            gold_list.append(gold)
            test_list.append(test_value)
    print(len(test_list))
    return gold_list, test_list


def add_space_before_punctuation(text):
    # List of punctuation marks
    punctuation_marks = string.punctuation

    # Iterate over each character in the text
    modified_text = ""
    for char in text:
        if char in punctuation_marks:
            modified_text += " " + char
        else:
            modified_text += char

    return modified_text


def add_space_before_punctuation(text):
    # List of punctuation marks
    punctuation_marks = string.punctuation

    # Iterate over each character in the text
    modified_text = ""
    for char in text:
        if char in punctuation_marks:
            modified_text += " " + char
        else:
            modified_text += char

    return modified_text


def add_space(reference_list, hypothesis_list):
    reference = []
    hypothesis = []
    for ref, hyp in zip(reference_list, hypothesis_list):
        # Preprocess sentences to keep punctuation as separate words
        reference.append(add_space_before_punctuation(ref))
        hypothesis.append(add_space_before_punctuation(hyp))
        # print(reference,hypothesis)
        # Calculate WER

    #     wer = jiwer.wer(reference, hypothesis)
    #     # print(wer)
    #     wer_scores.append(wer)
    # wer_mean=sum(wer_scores) / len(wer_scores)

    return reference, hypothesis


def get_detail_wer(gold_list, test_list):
    out = jiwer.process_words(
        gold_list,
        test_list,
    )
    print("wer: ", out.wer * 100)
    print("hits: ", out.hits)
    print("substitutions: ", out.substitutions)
    print("insertions: ", out.insertions)
    print("deletions: ", out.deletions)
    print("total words: ", out.hits + out.substitutions + out.deletions)


# def get_t_p_wer(substitutions, insertions,deletions,total_words_punc, P_WER):
#     t_p_wer=(substitutions+insertions+deletions)/total_words_punc
#     print("Topline of P-WER is ",t_p_wer*100)
#     print("Relative PER is ", P_WER-t_p_wer*100)


def get_c_p_er_fr_wer(wer, C_WER, P_WER, clean=True):
    if clean:
        total_words = 50082
        total_words_punt = 57312
        cased_words = 3849
        p_num = 9709
    else:
        total_words = 48488
        total_words_punt = 55896
        cased_words = 4231
        p_num = 9408
    error_words = wer * total_words
    error_c = C_WER * total_words
    cer = (error_c - error_words) / cased_words
    error_p = P_WER * total_words_punt
    per = (error_p - error_words) / p_num
    # print("Topline of P-WER is ",t_p_wer)
    print("Relative CER is ", cer)
    print("Relative PER is ", per)


def get_c_p_er_fr_wer_ami(wer, C_WER, P_WER):
    # if clean:
    total_words = 94006
    total_words_punt = 110266
    cased_words = 7626
    p_num = 16260
    error_words = wer * total_words
    error_c = C_WER * total_words
    cer = (error_c - error_words) / cased_words
    error_p = P_WER * total_words_punt
    per = (error_p - error_words) / p_num
    print("Relative CER is ", cer)
    print("Relative PER is ", per)


def get_c_p_er_fr_wer_cv(wer, C_WER, P_WER):
    # if clean:
    total_words = 11244
    total_words_punt = 13146
    cased_words = 1882
    p_num = 1902
    error_words = wer * total_words
    error_c = C_WER * total_words
    cer = (error_c - error_words) / cased_words
    error_p = P_WER * total_words_punt
    per = (error_p - error_words) / p_num
    print("Relative CER is ", cer)
    print("Relative PER is ", per)


def get_cased_word(text_list):
    capitalized_word_count = 0

    # Iterate through the list of text
    for text in text_list:
        # Split the text into words
        words = text.split()

        # Check each word in the text
        for word in words:
            # Check if the word is capitalized (first character is uppercase)
            if word.isalpha() and word[0].isupper():
                capitalized_word_count += 1
    print("Cased words number is ", capitalized_word_count)

Using cache found in /Users/ccui/.cache/torch/hub/snakers4_silero-models_master


In [4]:
gold_list, test_list = get_gold_hyp_lists(
    "/Users/ccui/Desktop/POC/dictation/CommonVoice/cv-en_test-2k_len4-10.txt",
    "cv-40.txt",
    ",",
)
get_cased_word(gold_list)
gold_list, test_list = add_space(gold_list, test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
gold_rc = jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp = jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp = jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

In [3]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/books-transcription/test-clean-rich-book.txt","clean_casePunc.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

2491
CP-WER
wer:  11.706840618132347
hits:  51117
substitutions:  3669
insertions:  495
deletions:  2548
total words:  57334
P-WER
wer:  9.423727631074057
hits:  52433
substitutions:  2346
insertions:  502
deletions:  2555
total words:  57334
C-WER
wer:  4.926147704590819
hits:  47752
substitutions:  2230
insertions:  120
deletions:  118
total words:  50100
wer:  2.189620758483034
hits:  49123
substitutions:  859
insertions:  120
deletions:  118
total words:  50100


In [4]:
get_c_p_er_fr_wer(wer=2.189620758483034,C_WER=4.926147704590819,P_WER=9.423727631074057,clean=True)

Relative CER is  35.606843989340106
Relative PER is  44.33330839074766


In [5]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/books-transcription/test-other-rich-book.txt","other_casePunc.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
# print("Total WER:", total_wer*100)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
print("WER")
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

2731
CP-WER
wer:  14.913896389549544
hits:  48195
substitutions:  5018
insertions:  614
deletions:  2708
total words:  55921
P-WER
wer:  12.39605872570233
hits:  49617
substitutions:  3582
insertions:  628
deletions:  2722
total words:  55921
C-WER
wer:  8.075123178098007
hits:  44820
substitutions:  3423
insertions:  230
deletions:  264
total words:  48507
WER
wer:  5.036386500917393
hits:  46296
substitutions:  1945
insertions:  232
deletions:  266
total words:  48507


In [6]:
get_c_p_er_fr_wer(wer=5.036386500917393,C_WER=8.075123178098007,P_WER=12.39605872570233,clean=False)

Relative CER is  34.82445379417008
Relative PER is  47.691941951039006


In [9]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/vasr/calculate_wer/ami_test_gold_process.txt","ihm_casePunc.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
# print("Total WER:", total_wer*100)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

9474
CP-WER
wer:  45.9960459253079
hits:  63081
substitutions:  26610
insertions:  3533
deletions:  20575
total words:  110266
P-WER
wer:  42.97698293218218
hits:  66524
substitutions:  23053
insertions:  3647
deletions:  20689
total words:  110266
C-WER
wer:  41.19098780928877
hits:  57268
substitutions:  22073
insertions:  1984
deletions:  14665
total words:  94006
wer:  37.465693679126865
hits:  60823
substitutions:  18465
insertions:  2037
deletions:  14718
total words:  94006


In [10]:
get_c_p_er_fr_wer_ami(wer=37.465693679126865,C_WER=41.19098780928877,P_WER=42.97698293218218)

Relative CER is  45.921846315237346
Relative PER is  74.840098400984
